In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
from pathlib import Path
import gzip

import s2gm
from datacube import Datacube
import odc.index

dc = Datacube()
cfg = s2gm.load_config('gmcfg.toml')

dss_pkl = Path(f'dss_{cfg.year}_{cfg.season}.pkl.gz')
tasks_pkl = Path(f'tasks_{cfg.year}_{cfg.season}.pkl.gz')

In [ ]:
%%time

if dss_pkl.exists():
    print('Load from pickle', end='')
    _dss = pickle.load(gzip.open(dss_pkl, 'rb'))
else:

    print(f'Querying DB: {cfg.period}')
    _dss = []
    for p in cfg.products:
        _dss.extend(dc.find_datasets_lazy(product=p, time=cfg.period))
        
    print('Pickling', end='')
    pickle.dump(_dss, gzip.open(dss_pkl, 'wb'))
    
print(' ...done')
print(f'Total of {len(_dss):,} datasets for {cfg.year}-{cfg.season}')

In [ ]:
%%time

tasks = s2gm.dss_to_tasks(_dss, cfg)

In [ ]:
list(t.region for t in tasks)[:10], set(t.region[0] for t in tasks)

In [ ]:
len(set(map(lambda t: t.uuid, tasks))), len(tasks)

In [ ]:
print(f"Saving to: {tasks_pkl}", end='')
pickle.dump(tasks, gzip.open(tasks_pkl, 'wb'))
print(" ... done")
!ls -lh *pkl.gz

----------------------------------------------------------------